# Diabetes prediction model using data science and machine learning

In [1]:
# Step 1: Import Libraries


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


from sklearn.svm import SVC
import os
import sys
from pathlib import Path
import logging
import warnings


# Step 2: Load Dataset

In [2]:
df = pd.read_csv(r'C:\Users\mahes\OneDrive - United Nations\Data Science course Naresh\3 Data Science\Projects\Diabetes Prediction\diabetes.csv')
df.head()
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

# Step 3: Clean & Preprocess Data

In [3]:
cols_with_zero = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
for col in cols_with_zero:
    df[col] = df[col].replace(0, np.nan)

df.fillna(df.mean(), inplace=True)

# Step 4: Train/Test Split

In [4]:
X = df.drop("Outcome", axis=1)
y = df["Outcome"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)


# Step 5: Feature Scaling

In [5]:
# ✔ Logistic Regression

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Logistic Regression with more iterations
log_reg = LogisticRegression(max_iter=2000, solver='lbfgs')
log_reg.fit(X_train, y_train)

y_pred_lr = log_reg.predict(X_test)



In [6]:
# ✔ Random Forest

rf = RandomForestClassifier(n_estimators=200, random_state=0)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)


In [7]:
# ✔ Support Vector Machine

svm_clf = SVC(kernel='rbf', probability=True)
svm_clf.fit(X_train, y_train)
y_pred_svm = svm_clf.predict(X_test)


In [8]:
# Step 7: Evaluate Models

def evaluate_model(name, y_true, y_pred):
    print(f"==== {name} ====")
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("ROC AUC:", roc_auc_score(y_true, y_pred))
    print(classification_report(y_true, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
    print("\n")


In [9]:
# Run Evaluations

evaluate_model("Logistic Regression", y_test, y_pred_lr)
evaluate_model("Random Forest", y_test, y_pred_rf)
evaluate_model("SVM", y_test, y_pred_svm)


==== Logistic Regression ====
Accuracy: 0.8116883116883117
ROC AUC: 0.7511433684629151
              precision    recall  f1-score   support

           0       0.84      0.91      0.87       107
           1       0.74      0.60      0.66        47

    accuracy                           0.81       154
   macro avg       0.79      0.75      0.76       154
weighted avg       0.81      0.81      0.81       154

Confusion Matrix:
 [[97 10]
 [19 28]]


==== Random Forest ====
Accuracy: 0.8051948051948052
ROC AUC: 0.7643666732948896
              precision    recall  f1-score   support

           0       0.85      0.87      0.86       107
           1       0.69      0.66      0.67        47

    accuracy                           0.81       154
   macro avg       0.77      0.76      0.77       154
weighted avg       0.80      0.81      0.80       154

Confusion Matrix:
 [[93 14]
 [16 31]]


==== SVM ====
Accuracy: 0.7727272727272727
ROC AUC: 0.7052097832571087
              precision    

In [ ]:
# Step 8: Improve Model (Optional)

In [10]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [4, 6, 8, 10]
}

grid = GridSearchCV(RandomForestClassifier(), param_grid, cv=5)
grid.fit(X_train, y_train)

print(grid.best_params_)
best_rf = grid.best_estimator_


{'max_depth': 6, 'n_estimators': 300}


In [11]:
# ep 9: Save Final Model

import joblib

joblib.dump(best_rf, "diabetes_model.pkl")
joblib.dump(scaler, "scaler.pkl")


['scaler.pkl']